In [ ]:
#!pip3 install --upgrade boto3   

# Mejores prácticas para diseñar prompts efectivos

In [16]:
import boto3
import json


In [17]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-east-1'
)

In [18]:
def api_bedrock(prompt):
    response = bedrock_runtime.converse(
    modelId="anthropic.claude-3-haiku-20240307-v1:0",
    messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
    inferenceConfig = {
          "temperature":1,
          "maxTokens":250,
          "topP":0.999
          }
          )

    try:
        result = response['output']['message']['content'][0]['text'] \
        + '\n--- Latency: ' + str(response['metrics']['latencyMs']) \
        + 'ms - Input tokens:' + str(response['usage']['inputTokens']) \
        + ' - Output tokens:' + str(response['usage']['outputTokens']) + ' ---\n'
        return result
    except Exception as e:
        print(e)
        result = "Output parsing error"
    print(result)
    return result

## Ser claro y conciso


In [ ]:
prompt1="Escribe un haiku sobre robots"
result = api_bedrock(prompt1)
print(result)

In [ ]:
prompt2="""¿Escribe un haiku sobre robots. 
            Omite el preámbulo y pasa directamente al poema."""
result = api_bedrock(prompt2)
print(result)

## Incluya contexto si es necesario


In [6]:
text = """Amazon Q Developer is a generative artificial intelligence (AI) 
powered conversational assistant that can help you understand, build, extend, 
and operate AWS applications. You can ask questions about AWS architecture, 
your AWS resources, best practices, documentation, support, and more. 
Amazon Q is constantly updating its capabilities so your questions get 
the most contextually relevant and actionable answers.

When used in an integrated development environment (IDE), Amazon Q provides 
software development assistance. Amazon Q can chat about code, provide inline 
code completions, generate net new code, scan your code for security vulnerabilities, 
and make code upgrades and improvements, such as language updates, debugging, and optimizations.

Amazon Q is powered by Amazon Bedrock, a fully managed service that makes 
foundation models (FMs) available through an API. The model that powers Amazon Q 
has been augmented with high quality AWS content to get you more complete, actionable, 
and referenced answers to accelerate your building on AWS."""

In [ ]:
prompt1="Resumeme el articulo: "+text
result = api_bedrock(prompt1)
print(result)

In [ ]:
prompt2="Proporcione un resumen de este artículo para utilizarlo en una publicación de blog: "+text
result = api_bedrock(prompt2)
print(result)

## USA XML tags

In [10]:
text = """Amazon Q Developer is a generative artificial intelligence (AI) 
powered conversational assistant that can help you understand, build, extend, 
and operate AWS applications. You can ask questions about AWS architecture, 
your AWS resources, best practices, documentation, support, and more. 
Amazon Q is constantly updating its capabilities so your questions get 
the most contextually relevant and actionable answers.

When used in an integrated development environment (IDE), Amazon Q provides 
software development assistance. Amazon Q can chat about code, provide inline 
code completions, generate net new code, scan your code for security vulnerabilities, 
and make code upgrades and improvements, such as language updates, debugging, and optimizations.

Amazon Q is powered by Amazon Bedrock, a fully managed service that makes 
foundation models (FMs) available through an API. The model that powers Amazon Q 
has been augmented with high quality AWS content to get you more complete, actionable, 
and referenced answers to accelerate your building on AWS."""
RED_SOCIAL = "linkedin"

In [ ]:
prompt=f""" Eres un experto en publicidad en redes sociales, te entregan el texto y la red social y creas la publicidad. 
<texto>{text}</texto>
<red_social>{RED_SOCIAL}</red_social>
"""
result = api_bedrock(prompt)
print(result)

## Zero-Shot

In [ ]:
prompt=f""" Clasifique el texto en neutro, negativo o positivo.
Texto: Creo que Mazatlán esta increíble.
Sentimiento:
"""
result = api_bedrock(prompt)
print(result)

## Few-Shot

In [ ]:
prompt=f""" Clasificar reseñas de restaurantes como positivas o negativas.

Reseña: La comida estaba deliciosa y el servicio fue excelente.
Clasificación: Positiva

Reseña: El lugar estaba sucio y la comida fría.
Clasificación: Negativa

Reseña: Precios razonables y ambiente agradable.
Clasificación:
"""
result = api_bedrock(prompt)
print(result)

## Chain-of-thought prompting
## Zero-shot

In [ ]:
prompt=f"""Problema:
En una fiesta de la Pycon, hay 3 pizzas grandes. Cada pizza está cortada en 8 rebanadas. Si hay 12 invitados y cada uno come 2 rebanadas, ¿cuántas rebanadas de pizza sobrarán?

Instrucción:
Resuelve este problema pensando paso a paso. Muestra tu razonamiento en cada etapa.

"""
result = api_bedrock(prompt)
print(result)

## Chain-of-thought prompting
## Few-shot

In [ ]:
prompt=f"""Tarea: Calcular cuántos tacos puede comer un programador en una noche de coding.
Datos: 3 horas de coding, 1 bug resuelto
Pensamiento: 
1. Base: 3 tacos/hora x 3 horas = 9 tacos 
2. Bonus por bug: +2 tacos
Total: 11 tacos
Datos: 2 horas de coding, 1 feature completada
Pensamiento: 
1. Base: 3 tacos/hora x 2 horas = 6 tacos
2. Bonus por feature: +3 tacos
Total: 9 tacos

Datos: 4 horas de coding, 2 bugs resueltos, 1 crash del sistema:

"""
result = api_bedrock(prompt)
print(result)

## Utilice directivas para el tipo de respuesta deseado


In [ ]:
prompt1="¿Cuál es la capital de Nueva York? Proporcione la respuesta en una oración completa."
result = api_bedrock(prompt1)
print(result)

In [ ]:
prompt2="¿Cuál es la capital de Nueva York?"
result = api_bedrock(prompt2)
print(result)

## Considere el resultado en el mensaje

In [ ]:
prompt1="Calcula el área de un círculo con un radio de 3 pulgadas"
api_bedrock(prompt1)

In [ ]:
prompt2="Calcula el área de un círculo con un radio de 3 pulgadas (7,5 cm). Redondea tu respuesta al número entero más cercano."
api_bedrock(prompt2)

## Iniciar mensajes con una pregunta

In [ ]:
evento = """ En septiembre de 1666, un pequeño incendio comenzó en la panadería del panadero Thomas Farriner en Pudding Lane, en Londres. Sin embargo, debido a las condiciones secas y al viento fuerte, el fuego se extendió rápidamente a través de las casas de madera apiñadas en la ciudad. Dentro de pocas horas, el fuego se había extendido desde Pudding Lane hasta la catedral de San Pablo. Al día siguiente, el fuego había arrasado la mayor parte de la ciudad dentro de las murallas. Durante casi una semana, el fuego ardió sin control. Solo se detuvo cuando el viento cambió de dirección y los esfuerzos para demoler edificios crearon calles de fuego para detener su propagación.
Cuando finalmente terminó, el Gran Incendio de Londres había destruido 13,200 casas, 87 iglesias parroquiales, St. Paul's Cathedral y muchos edificios gubernamentales. Se estima que la pérdida de vidas humanas fue de solo unas pocas docenas, pero decenas de miles de londinenses se quedaron sin hogar. Fue uno de los peores desastres en la historia de Inglaterra. El incendio tuvo un impacto duradero en Londres y en Inglaterra. Sirvió para impulsar la reconstrucción de Londres con amplias calles y edificios de piedra en lugar de madera, reduciendo así el riesgo de futuros incendios a gran escala. También llevó a la creación del London Building Acts, uno de los primeros códigos de construcción del mundo."""

In [ ]:
prompt1= "¿Por qué ocurrió el evento en los tags <evento></evento>? Explícalo en tres frases. <evento>"+evento+"</evento>"

api_bedrock(prompt1)

In [ ]:
prompt2= "Resume  el evento en los tags <evento></evento>. <evento>"+evento+"</evento>"

api_bedrock(prompt2)

## Proporcione una respuesta de ejemplo (one shot)

In [ ]:
review_negativo =  """Compré el nuevo teléfono X pero ha sido una decepción total. La batería dura apenas unas horas y la cámara toma fotos borrosas. Además se cuelga constantemente. No puedo recomendar este producto."""

In [ ]:
prompt1= "Determine el sentimiento de esta publicación en las redes sociales:"+review_negativo
api_bedrock(prompt1)

In [ ]:
prompt2 = f"""Determine el sentimiento de la siguiente publicación en las redes sociales utilizando estos ejemplos:
post: “Muy buen producto, me encanta.” sentimiento: Positivo
post: “Odio cuando se agota la batería de mi teléfono.” sentimiento:  Negativo
post: "{review_negativo}" sentimiento:  """
api_bedrock(prompt2)